# Python Programming Practice
## Chapter 08. 정규 표현식 Regular expression

### 08-1. 정규표현식 살펴보기

In [1]:
# 주민등록번호를 포함하는 텍스트에 포함된 모든 주민등록번호의 뒷자리를 *문자로 변경

def number_to_star(data):
    result = []
    for line in data.split("\n"):
        word_result = []
        for word in line.split(" "):
            # 뒷자리를 *로 변경
            if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
                word = word[:6] + "-" + "*" * 7
            # " "로 구분된 모든 단어 저장
            word_result.append(word)
        # 단어들을 묶어서 result에 저장
        result.append(" ".join(word_result))

    print("\n".join(result))

data = """
park 800905-1049118
kim 980901-2578123
"""

number_to_star(data)


park 800905-*******
kim 980901-*******



In [3]:
# regular expression 사용
import re

pat = re.compile(r"(\d{6})[-]\d{7}")
print(pat.sub(r"\g<1>-*******", data))


park 800905-*******
kim 980901-*******



### 08-2. 정규표현식 시작하기

#### 정규 표현식의 기초, 메타 문자
> . ^ $ + ? {} [] \ | ()

In [13]:
import re

def print_match(reg_pattern, text):
    p = re.compile(reg_pattern)

    print(p.match(text))

In [ ]:
# 1. []: character class

# [abc]: a, b, c중 하나라도 매치
print_match(r"[abc]", "a")
print_match(r"[abc]", "before")
print_match(r"[abc]", "dude")

<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='b'>
None


In [26]:
# [a-zA-Z]: 모든 문자, [0-9]: 모든 숫자
print_match(r"\d", "34") # \d == [0-9]
print_match(r"\D", "str") # \D == [^0-9]
print_match(r"\s", "a\nb") # \s == [ \t\n\r\f\v], \S == [ ^\t\n...\v]
print_match(r"\s", "\n") 
print_match(r"\w", "my 8 goals") # \w == [a-zA-Z0-9], white space 전까지 검색
print_match(r"\W", "my 8 goals") # \W == [^a-zA-Z0-9], white space 전까지 검색

<re.Match object; span=(0, 1), match='3'>
<re.Match object; span=(0, 1), match='s'>
None
<re.Match object; span=(0, 1), match='\n'>
<re.Match object; span=(0, 1), match='m'>
None


In [32]:
# 2. .(dot): \n을 제외한 모든 문자 하나
print_match(r"a.b", "a1b") # O
print_match(r"a.b", "aab") # O
print_match(r"a.b", "abc") # X
print_match(r"a.b", "avvb") # X


<re.Match object; span=(0, 3), match='a1b'>
<re.Match object; span=(0, 3), match='aab'>
None
None


In [33]:
# 3. *: {0, inf} 반복
print_match(r"ca*t", "ct") # O
print_match(r"ca*t", "cat") # O
print_match(r"ca*t", "caat") # O
print_match(r"ca*t", "cdt") # X

<re.Match object; span=(0, 2), match='ct'>
<re.Match object; span=(0, 3), match='cat'>
<re.Match object; span=(0, 4), match='caat'>
None


In [35]:
# 4. + : {1, inf} 반복
print_match(r"ca+t", "ct") # X
print_match(r"ca+t", "cat") # O
print_match(r"ca+t", "caat") # O
print_match(r"ca+t", "cdt") # X

None
<re.Match object; span=(0, 3), match='cat'>
<re.Match object; span=(0, 4), match='caat'>
None


In [ ]:
# 5. {m,n}: m <= i <= n, i번 반복 / {m}: m번 반복
print_match(r"ca{2,5}t", "cat") # X
print_match(r"ca{2,5}t", "caat") # O
print_match(r"ca{2,5}t", "caaaaat") # O
print_match(r"ca{2,5}t", "cbt") # X
print_match(r"ca{2}t", "cat") # X
print_match(r"ca{2}t", "caat") # O


None
<re.Match object; span=(0, 4), match='caat'>
<re.Match object; span=(0, 7), match='caaaaat'>
None
None
<re.Match object; span=(0, 4), match='caat'>


In [39]:
# 6. ? : {0, 1}, 있거나 없어도 됨
# r"ab?c" == "abc" or "ac"
print_match(r"ab?c", "abc") # O
print_match(r"ab?c", "ac") # O
print_match(r"ab?c", "arc") # X

<re.Match object; span=(0, 3), match='abc'>
<re.Match object; span=(0, 2), match='ac'>
None


#### 정규식을 이용한 문자열 검색

`p = re.compile(r"regular expression pattern")`

- 컴파일 된 패턴 객체를 사용하여 문자열 검색 수행

1. `match()`: 문자열 처음부터 정규식패턴과 매치되는지 조사, return match obj or None
2. `search()`: 문자열 전체를 검색하여 정규식패턴과 매치되는지 조사, return match obj or None
3. `findall()`: 정규식패턴과 매치되는 모든 문자열(substring)을 list로 return, item은 string
4. `finditer()`: 정규식패턴과 매치되는 모든 문자열(substring)을 iterator 객체로 return, item은 match 객체

In [1]:
import re

# 영어 소문자가 하나 이상 반복
p = re.compile(r"[a-z]+") 

In [ ]:
# 1. match(): 문자열 처음부터 조사 
m = p.match("python")
print(m)

<re.Match object; span=(0, 6), match='python'>


In [ ]:
m = p.match("3 python") # while space 이전까지 조사
print(m)

None


In [5]:
# 2. search(): 문자열 전체 조사
m = p.search("python")
print(m)

m = p.search("3 python") # white space 이후 문자열까지 모두 조사
print(m)

<re.Match object; span=(0, 6), match='python'>
<re.Match object; span=(2, 8), match='python'>


In [ ]:
# 3. findall(): match되는 모든 문자열(str) list로 return
result = p.findall("life is too short, I Like Python")
print(result)

['life', 'is', 'too', 'short', 'ike', 'ython']


In [ ]:
# 4. finditer(): match되는 모든 문자열(match 객체) iterator로 return
result = p.finditer("life is too short")
print(result)

for item in result:
    print(item)

<re.Match object; span=(0, 4), match='life'>
<re.Match object; span=(5, 7), match='is'>
<re.Match object; span=(8, 11), match='too'>
<re.Match object; span=(12, 17), match='short'>


#### `match` 객체의 메서드
1. `group()`: 매치된 문자열 return
2. `start()`: 매치된 문자열 시작 위치 return
3. `end()`: 매치된 문자열 끝 위치 return
4. `span()`: 매치된 문자열 (start, end) 튜플 리턴

In [11]:
p = re.compile(r"[a-z]+")
m = p.match("python")

print(m.group())
print(m.start())
print(m.end())
print(m.span())

python
0
6
(0, 6)


In [12]:
m = p.search("3 python")
print(m.group())
print(m.start())
print(m.end())
print(m.span())

python
2
8
(2, 8)


In [14]:
# 모듈 단위로 수행하기
p = re.compile(r"[a-z]+")
m = p.match("python")

print(m)

<re.Match object; span=(0, 6), match='python'>


In [16]:
m = re.match(r"[a-z]+", "python")
print(m)

<re.Match object; span=(0, 6), match='python'>


#### 컴파일 옵션
1. `DOTALL` == `S`: .(dot)이 \n을 포함해 모든 문자와 매칭
2. `IGNORECASE` == `I`: 대소문자 관계없이 매칭
3. `MULTILINE` == `M`: 여러 줄과 매칭, `^`, `$` 메타문자 각 줄마다 적용
4. `VERBOSE` == `X`: verbose 모드 사용, 정규식 내부 white space 무시하여 주석등 추가 가능

In [ ]:
# 1. DOTALL, S : .(dot)이 \n 포함 
# > 여러줄 문자열에서 줄바꿈과 상관없이 검색할 때 사용
p = re.compile(r"a.b")
m = p.match("a\nb")

print(m)

None


In [20]:
p = re.compile(r"a.b", re.DOTALL)
m = p.match("a\nb")

print(m)

<re.Match object; span=(0, 3), match='a\nb'>


In [21]:
# 2. IGNORECASE, I: 대소문자 구분 없이 매칭
p = re.compile(r"[a-z]+", re.I)
print(p.match("python"))
print(p.match("Python"))
print(p.match("PYTHON"))

<re.Match object; span=(0, 6), match='python'>
<re.Match object; span=(0, 6), match='Python'>
<re.Match object; span=(0, 6), match='PYTHON'>


In [ ]:
# 3. MULTILINE, M: 여러줄과 매칭
# ^, $ 메타 문자를 각 줄마다 적용
import re

# python으로 시작, 화이트 스페이스 이후 어떤 문자든 와도 됨
p = re.compile(r"^python\s\w+")

data = '''python one
life is too short
python two
you need python
python three
'''

print(p.findall(data))

['python one']


In [4]:
# 각 라인별로 매칭
p = re.compile(r"^python\s\w+", re.M)

print(p.findall(data))

['python one', 'python two', 'python three']


In [11]:
# 4. VERBOSE, X: 정규식 이해할 수 있도록 변환 가능, 정규식 안의 white space 무시

# 이해하기 어려움
charref1 = re.compile(r"&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);")

In [12]:
# VERBOSE 옵션 적용, white space 무시, 주석 달기 및 라인 구분 가능
charref2 = re.compile(
r"""
&[#]    # Start of a numeric entity reference
(
0[0-7]+     # Octal form
| [0-9]+    # Decimal form
| x[0-9a-fA-F]+ # Hecadecimal form
)
;                         
""", re.VERBOSE)

&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);

&[#]    # Start of a numeric entity reference
(
0[0-7]+     # Octal form
| [0-9]+    # Decimal form
| x[0-9a-fA-F]+ # Hecadecimal form
)
;                         



### 08-3. 강력한 정규 표현식의 세계로

#### 문자열 소비가 없는 메타 문자